In [0]:
# -*- coding: utf-8 -*-

#Check Cuda Version
#!nvcc --version

#!pip uninstall pytorch
#!pip uninstall pytorch-nightly
#!pip uninstall torchvision

#Download Cuda 9.2 and PyTorch 0.4.1
#!git clone https://gist.github.com/f7b7c7758a46da49f84bc68b47997d69.git
#!bash /content/f7b7c7758a46da49f84bc68b47997d69/pytorch041_cuda92_colab.sh



#!pip3 install https://download.pytorch.org/whl/cu100/torch-1.0.1.post2-cp36-cp36m-linux_x86_64.whl
#!pip3 install torchvision

#!pip3 install --upgrade torch torchvision

import torch
!nvcc --version
print(torch.__version__)
print(torch.cuda.get_device_name(0))


from __future__ import print_function, division

import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import cv2
import torch.hub
from PIL import Image, ImageOps, ImageEnhance, PILLOW_VERSION

#import fastai



class adjust_brightness(object):
    def __init__(self, brightness_factor):
      self.brightness_factor = brightness_factor
    
    def __call__(self, img):
      enhancer = ImageEnhance.Brightness(img)
      img = enhancer.enhance(self.brightness_factor)
      return img    

  
#print(torchvision.models.resnet.model_urls)
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_transforms2 = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        #transforms.ColorJitter(brightness=0.6, contrast=0, saturation=0, hue=0),
        #transforms.RandomGrayscale(p=1),
        adjust_brightness(0.65),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        adjust_brightness(0.9),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/content/drive/My Drive/Colab Notebooks/notebook/catdog_data'
#data_dir = '/content/drive/My Drive/catdog_data'
save_path1 = '/content/drive/My Drive/Colab Notebooks/notebook/CNN_dogcat0810_SE-ResNet50_nondict5.pt'
save_path2 = '/content/drive/My Drive/Colab Notebooks/notebook/CNN_dogcat0810_SE-ResNet50_dict5.pt'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
image_datasets2 = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms2[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=24,
                                             shuffle=True, num_workers=16)
              for x in ['train', 'val']}
dataloaders2 = {x: torch.utils.data.DataLoader(image_datasets2[x], batch_size=24,
                                             shuffle=True, num_workers=16)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



######################################################################
# Visualize a few images
# ^^^^^^^^^^^^^^^^^^^^^^
# Let's visualize a few training images so as to understand the data
# augmentations.

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

# imshow(out, title=[class_names[x] for x in classes])


######################################################################
# Training the model
# ------------------
#
# Now, let's write a general function to train a model. Here, we will
# illustrate:
#
# -  Scheduling the learning rate
# -  Saving the best model
#
# In the following, parameter ``scheduler`` is an LR scheduler object from
# ``torch.optim.lr_scheduler``.


def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    print('Train start time is {}'.format(time.strftime('%c', time.localtime(since+32400))))
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    used_train_data_num = 0
    #print(used_train_data_num)
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                #used_train_data_num += 1

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                        
            #print(used_train_data_num)

            
            for inputs, labels in dataloaders2[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        #optimizer.step()
                used_train_data_num += 1

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / 2 / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / 2 / dataset_sizes[phase]

            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            #print(used_train_data_num)
            time_elapsed = time.time() - since
            print('Time elapsed is {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

            
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


######################################################################
# Visualizing the model predictions
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#
# Generic function to display predictions for a few images
#

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

######################################################################
# Finetuning the convnet
# ----------------------
#
# Load a pretrained model and reset final fully connected layer.
#

"""Original Model Used"""
"""with pretrained=Flase, you'll need much, much 
more epochs(often in hundreds) to get a good result"""
#model_ft = models.resnet18(pretrained=True)

# New Model Used
#model_ft = models.resnet50(pretrained=True)

#New Model test of SE-ResNet-50
#https://github.com/moskomule/senet.pytorch
#https://github.com/Xingxiangrui/various_pyTorch_network_structure/blob/master/senet_and_pretrained.py

model_ft = torch.hub.load('moskomule/senet.pytorch', 'se_resnet50', pretrained=True,)


num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()



def testing_dataloader2(model, num_images=10):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders2['train']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                #ax = plt.subplot(num_images//2, 2, images_so_far)
                #ax.axis('off')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        #model.train(mode=was_training)




# Observe that all parameters are being optimizedCNN_dogcat0810.pt

# Original Optimizer Used
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.002, momentum=0.9)

# New Optimizer Used_Performance Very Unsatisfactory
#optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.00002, betas = (0.9, 0.999), eps=1e-8)

# Decay LR by a factor of 0.1 every 4 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=8, gamma=0.1)

"""Use fixed LR for comparison"""

#learn = create_cnn(image_datasets, models.resnet50, metrics=accuracy)
#learn.lr_find()
#learn.recorder.plot()

######################################################################
# Train and evaluate
# ^^^^^^^^^^^^^^^^^^
#
# It should take around 15-25 min on CPU. On GPU though, it takes less than a
# minute.
#

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=30)



######################################################################
#


#testing_dataloader2(model_ft)




#visualize_model(model_ft)

# To resolve errors in CNN_test
torch.save(model_ft.state_dict(), save_path2)

# Original code for saving
torch.save(model_ft, save_path1)


print('Saved model')

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
1.3.0+cu100
Tesla K80


Downloading: "https://github.com/moskomule/senet.pytorch/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/moskomule/senet.pytorch/releases/download/archive/seresnet50-60a8950a85b2b.pkl" to /root/.cache/torch/checkpoints/seresnet50-60a8950a85b2b.pkl
100%|██████████| 107M/107M [00:04<00:00, 27.9MB/s]


Train start time is Sun Oct 20 17:47:17 2019
Epoch 0/29
----------


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train Loss: 0.1104 Acc: 0.9537
Time elapsed is 20m 29s
val Loss: 0.0886 Acc: 0.9647
Time elapsed is 27m 25s
Epoch 1/29
----------
train Loss: 0.0825 Acc: 0.9664
Time elapsed is 45m 50s
val Loss: 0.0708 Acc: 0.9740
Time elapsed is 48m 54s
Epoch 2/29
----------
train Loss: 0.0699 Acc: 0.9714
Time elapsed is 67m 20s
val Loss: 0.0635 Acc: 0.9785
Time elapsed is 70m 24s
Epoch 3/29
----------
train Loss: 0.0635 Acc: 0.9741
Time elapsed is 89m 2s
val Loss: 0.0502 Acc: 0.9837
Time elapsed is 92m 9s
Epoch 4/29
----------
train Loss: 0.0599 Acc: 0.9755
Time elapsed is 110m 25s
val Loss: 0.0851 Acc: 0.9679
Time elapsed is 113m 27s
Epoch 5/29
----------
train Loss: 0.0607 Acc: 0.9752
Time elapsed is 131m 48s
val Loss: 0.0492 Acc: 0.9842
Time elapsed is 134m 51s
Epoch 6/29
----------
train Loss: 0.0566 Acc: 0.9765
Time elapsed is 153m 17s
val Loss: 0.0637 Acc: 0.9787
Time elapsed is 156m 20s
Epoch 7/29
----------
train Loss: 0.0501 Acc: 0.9800
Time elapsed is 174m 34s
val Loss: 0.0610 Acc: 0.9798
T

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist

Saved model


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
